# Chunk External Properties Feature Example

This notebook demonstrates how to use external properties to add business-specific attributes to chunk nodes in the graph database.

## Overview

The external properties feature allows you to:
- Add any number of business properties from source metadata to chunks
- Query and filter chunks by multiple dimensions
- Support complex multi-attribute filtering
- Enable advanced analytics and reporting

## Setup

If you haven't already, install the toolkit and dependencies using the [Setup](./00-Setup.ipynb) notebook.

In [ ]:
env_content = """AWS_REGION="us-east-1"
EMBEDDINGS_MODEL="cohere.embed-english-v3"
EMBEDDINGS_DIMENSIONS=1024
EXTRACTION_MODEL="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
RESPONSE_MODEL="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
GRAPH_STORE="graph-store-endpoint:port"
VECTOR_STORE="postgres-endpoint"
"""

with open('.env', 'w') as f:
    f.write(env_content)
print("Updated .env")

In [ ]:
%reload_ext dotenv
%dotenv

import os
from pathlib import Path

from graphrag_toolkit.lexical_graph import GraphRAGConfig, LexicalGraphIndex, set_logging_config
from graphrag_toolkit.lexical_graph.storage import GraphStoreFactory, VectorStoreFactory
from llama_index.core import SimpleDirectoryReader

set_logging_config('INFO')

print(f"Graph Store: {os.environ.get('GRAPH_STORE', 'Not set')}")
print(f"Vector Store: {os.environ.get('VECTOR_STORE', 'Not set')}")

## Configure Document Source Folder

Set the path to your local folder containing documents to index.

In [ ]:
# Set the path to your documents folder
# Change this to point to your local folder
DOCUMENTS_FOLDER = "./data"

# Create the folder if it doesn't exist (for demo purposes)
Path(DOCUMENTS_FOLDER).mkdir(parents=True, exist_ok=True)

print(f"Documents folder: {os.path.abspath(DOCUMENTS_FOLDER)}")

## Configure External Properties

Define which metadata fields should be added as properties on chunk nodes.

In [ ]:
# Configure external properties - these will be added to chunk nodes
# Format: {'chunk_property_name': 'metadata_key'}
GraphRAGConfig.chunk_external_properties = {
    'file_name': 'file_name',
    'file_path': 'file_path',
    'file_type': 'file_type',
    'article_code': 'article_id',
    'department': 'department'
}

print("External properties configured:")
for prop, key in GraphRAGConfig.chunk_external_properties.items():
    print(f"  chunk.{prop} ← metadata['{key}']")

## Load Documents from Local Folder

In [ ]:
# Define a function to add custom metadata to documents
def add_custom_metadata(file_path: str) -> dict:
    """Add custom metadata based on file name or path."""
    filename = os.path.basename(file_path)
    
    # Example: derive metadata from filename
    metadata = {
        'file_name': filename,
        'file_path': file_path,
        'file_type': os.path.splitext(filename)[1].lstrip('.'),
        'article_id': os.path.splitext(filename)[0].lstrip('.')
    }
    
    return metadata

# Load documents from the folder
reader = SimpleDirectoryReader(
    input_dir=DOCUMENTS_FOLDER,
    recursive=True,
    file_metadata=add_custom_metadata
)

documents = reader.load_data()

print(f"Loaded {len(documents)} documents from {DOCUMENTS_FOLDER}")
print("\nDocuments with metadata:")
for doc in documents:
    print(f"  - {doc.metadata.get('file_name', 'N/A')}")
    print(f"    article_id: {doc.metadata.get('article_id', 'N/A')}")
    print(f"    department: {doc.metadata.get('department', 'N/A')}")

## Build the Graph with External Properties

In [ ]:
# Build the graph - external properties will be added to chunk nodes
with (
    GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE']) as graph_store,
    VectorStoreFactory.for_vector_store(os.environ['VECTOR_STORE']) as vector_store
):
    graph_index = LexicalGraphIndex(
        graph_store, 
        vector_store
    )
    
    # Extract and build - this will add external properties to chunks
    graph_index.extract_and_build(documents, show_progress=True)

print('\nGraph build complete!')
print('Chunks now have external properties from document metadata.')

## Query Chunks by External Properties

In [ ]:
# Verify that external properties are stored on chunk nodes
with GraphStoreFactory.for_graph_store(os.environ['GRAPH_STORE']) as graph_store:
    
    print("All chunks with their external properties:")
    print("=" * 70)
    
    query = """
    MATCH (chunk:`__Chunk__`)
    WHERE chunk.file_name IS NOT NULL
    RETURN chunk.chunkId as chunkId,
           chunk.file_name as file_name,
           chunk.article_code as article_code,
           chunk.department as department
    ORDER BY chunk.file_name
    """
    
    results = graph_store.execute_query(query)
    for r in results:
        print(f"File: {r.get('file_name', 'N/A')}")
        print(f"article_id: {r.get('article_code', 'N/A')}")
        print()

## Summary

This notebook demonstrated:

1. **Loading documents from a local folder** using `SimpleDirectoryReader`
2. **Adding custom metadata** via the `file_metadata` callback
3. **Configuring external properties** via `GraphRAGConfig.chunk_external_properties`
4. **Building the graph** with documents that have metadata
5. **Querying chunks** using the external properties for filtering